In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
df1 = pd.read_parquet('../../data/Train_X/2023_03_14_data.pq', engine='pyarrow')
df2 = pd.read_parquet('../../data/Train_X/2023_03_15_data.pq', engine='pyarrow')
df3 = pd.read_parquet('../../data/Eval_X/2023_03_16_data.pq', engine='pyarrow')
df1.head()

,ProzessData_ActData_AB1_Analogs_DX1_MassPressure,ProzessData_ActData_AB1_Analogs_GY1_MassLevelTank,ProzessData_ActData_AB1_Current_DV1_Scraper,ProzessData_ActData_AB1_Current_DW1_RiserPumpFwd,ProzessData_ActData_AB1_Speed_DV1_Scraper,ProzessData_ActData_AB1_Speed_DW1_RiserPumpFwd,ProzessData_ActData_AB1_Temperature_DP1_MassHeatingStage,ProzessData_ActData_AB1_Temperature_DP1_WaterHeatingStage,ProzessData_ActData_AB1_Temperature_DQ1_MassCoolingStage,ProzessData_ActData_AB1_Temperature_DQ1_WaterCoolingStage,ProzessData_ActData_AB1_Temperature_DR1_WaterMixingStage,ProzessData_ActData_AB1_Temperature_DS1_WaterPipe,ProzessData_ActData_AB1_Temperature_DU1_WaterTank,ProzessData_ActData_AB1_Temperature_DX1_MassInfeed
2023-03-14 01:00:00+00:00,0.92,63.349998,0.0,0.0,0.0,0.0,41.299999,42.000000,41.700001,42.0,42.0,41.849998,42.099998,41.900002
2023-03-14 01:00:01+00:00,0.93,63.340000,0.0,0.0,0.0,0.0,41.299999,42.000000,41.700001,42.0,42.0,41.849998,42.099998,41.900002
2023-03-14 01:00:02+00:00,0.93,63.340000,0.0,0.0,0.0,0.0,41.299999,42.000000,41.700001,42.0,42.0,41.849998,42.000000,41.900002
2023-03-14 01:00:03+00:00,0.92,63.349998,0.0,0.0,0.0,0.0,41.299999,41.900002,41.700001,42.0,42.0,41.849998,42.000000,41.900002
2023-03-14 01:00:04+00:00,0.92,63.340000,0.0,0.0,0.0,0.0,41.299999,41.900002,41.700001,42.0,42.0,41.849998,42.000000,41.900002
